In [1]:
import torch
import torchvision

In [4]:
!pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.2 MB 2.1 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.2 MB 831.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 811.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 811.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 686.8 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 681.0 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 681.0 kB/s eta 0:00:02
   ---------- ----------------------------- 0.3/1.2 MB 656.4 kB/s eta 0:00:02
   --

  You can safely remove it manually.


   -------- ------------------------------ 34.6/159.8 MB 424.0 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 423.5 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 422.9 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.5 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 422.7 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.8 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.8/159.8 MB 424.0 kB/s eta 0:04:55
   -------- ------------------------------ 34.8/159.8 MB 423.8 kB/s eta 0:04:56
   -------- ------------------------------ 34.8/159.8 MB 424.6 kB/s eta 0:04:55
   -------- ----------------------------

In [38]:
!python run.py --image_name 009835

iou co sai khong: 0.145728
1
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21027
while terminal step:1
iou co sai khong: 0.1909502840002553
1
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21273
while terminal step:2
iou co sai khong: 0.15325543223385962
2
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21316
while terminal step:3
iou co sai khong: 0.1987720887796286
3
[0. 0. 0. ... 0. 0. 0.]
(25088,)
20975
while terminal step:4
iou co sai khong: 0.25573731737488564
3
[0. 0. 0. ... 0. 0. 0.]
(25088,)
20854
while terminal step:5
iou co sai khong: 0.32575480777654975
4
[0. 0. 0. ... 0. 0. 0.]
(25088,)
20861
while terminal step:6
iou co sai khong: 0.23930635373733694
1
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21281
while terminal step:7
iou co sai khong: 0.16053824966915545
0
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21086
while terminal step:8
iou co sai khong: 0.2039157870755838
5
[0. 0. 0. ... 0. 0. 0.]
(25088,)
21086
while terminal step:8
iou co sai khong: 0.2039157870755838
4
[0. 0. 0. ... 0. 0. 0.]
(25088,)
20890
while terminal step:9
iou c

c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import os
import numpy as np
from PIL import Image
import torch

In [22]:
import run

In [3]:
path_voc_test = r"e:\msrlOB-1\archive (4)\VOCtest_06-Nov-2007\VOC2007"

In [4]:
import torch
import torchvision
from models.Q_net import Q_zoom, Q_refine
from data import load_images_names_in_data_set, get_bb_of_gt_from_pascal_xml_annotation
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse
import os
from PIL import Image, ImageDraw
from utils import cal_iou, reward_func
import time

# hyper-parameters
BATCH_SIZE = 100
LR = 1e-6
GAMMA = 0.9
MEMORY_CAPACITY = 1000
Q_NETWORK_ITERATION = 100
epochs = 500
NUM_ACTIONS = 6
his_actions = 4
subscale = 3/4
NUM_STATES = 7*7*512+his_actions*NUM_ACTIONS

In [5]:

def init_process(image, transform=None):
    # image.show()
    # time.sleep(5)
    if transform:
        image = transform(image)
    return image.unsqueeze(0)


def inter_process(image, bbx, transform=None):
    (left, upper, right, lower) = (bbx[0], bbx[2], bbx[1], bbx[3])
    image_crop = image.crop((left, upper, right, lower))
    # image_crop.show()
    # time.sleep(5)
    if transform:
        image_crop = transform(image_crop)
    return image_crop.unsqueeze(0)


def update_bbx(bbx, action):
    new_bbx = np.zeros(4)
    if action == 0:  # top left
        new_bbx[0] = bbx[0]  # x1
        new_bbx[1] = bbx[0] + (bbx[1]-bbx[0]) * subscale  # x2
        new_bbx[2] = bbx[2]  # y1
        new_bbx[3] = bbx[2] + (bbx[3]-bbx[2]) * subscale  # y2
    elif action == 1:  # top right
        new_bbx[0] = bbx[1] - (bbx[1]-bbx[0]) * subscale  # x1
        new_bbx[1] = bbx[1]  # x2
        new_bbx[2] = bbx[2]  # y1
        new_bbx[3] = bbx[2] + (bbx[3]-bbx[2]) * subscale  # y2
    elif action == 2:  # lower left
        new_bbx[0] = bbx[0]  # x1
        new_bbx[1] = bbx[0] + (bbx[1]-bbx[0]) * subscale  # x2
        new_bbx[2] = bbx[3] - (bbx[3]-bbx[2]) * subscale  # y1
        new_bbx[3] = bbx[3]  # y2
    elif action == 3:  # lower right
        new_bbx[0] = bbx[1] - (bbx[1]-bbx[0]) * subscale  # x1
        new_bbx[1] = bbx[1]  # x2
        new_bbx[2] = bbx[3] - (bbx[3]-bbx[2]) * subscale  # y1
        new_bbx[3] = bbx[3]  # y2
    elif action == 4:  # center
        new_bbx[0] = (bbx[0]+bbx[1])/2-(bbx[1]-bbx[0]) * subscale/2  # x1
        new_bbx[1] = (bbx[0]+bbx[1])/2+(bbx[1]-bbx[0]) * subscale/2  # x2
        new_bbx[2] = (bbx[2]+bbx[3])/2-(bbx[3]-bbx[2]) * subscale/2  # y1
        new_bbx[3] = (bbx[2]+bbx[3])/2+(bbx[3]-bbx[2]) * subscale/2  # y2
    elif action == 5:
        new_bbx = bbx
    return new_bbx

# def update_bbx(bbx, action):
#     new_bbx = np.zeros(4)
#     if action == 0:  # left
#         new_bbx[0] = bbx[0]  # x1
#         new_bbx[1] = bbx[0] + (bbx[1]-bbx[0]) * subscale  # x2
#         new_bbx[2] = bbx[2]  # y1
#         new_bbx[3] = bbx[3] #+ (bbx[3]-bbx[2]) * subscale  # y2
#     elif action == 1:  # right
#         new_bbx[0] = bbx[1] - (bbx[1]-bbx[0]) * subscale  # x1
#         new_bbx[1] = bbx[1]  # x2
#         new_bbx[2] = bbx[2]  # y1
#         new_bbx[3] = bbx[3] # + (bbx[3]-bbx[2]) * subscale  # y2
#     elif action == 2:  # bot 
#         new_bbx[0] = bbx[0]  # x1
#         new_bbx[1] = bbx[1] #+ (bbx[1]-bbx[0]) * subscale  # x2
#         new_bbx[2] = bbx[3] - (bbx[3]-bbx[2]) * subscale  # y1
#         new_bbx[3] = bbx[3]  # y2
#     elif action == 3:  # top
#         new_bbx[0] = bbx[0]  #- (bbx[1]-bbx[0]) * subscale  # x1
#         new_bbx[1] = bbx[1]  # x2
#         new_bbx[2] = bbx[2]  # y1
#         new_bbx[3] = bbx[2] + (bbx[3]-bbx[2]) * subscale  # y2
#     elif action == 4:  # center
#         new_bbx[0] = (bbx[0]+bbx[1])/2-(bbx[1]-bbx[0]) * subscale/2  # x1
#         new_bbx[1] = (bbx[0]+bbx[1])/2+(bbx[1]-bbx[0]) * subscale/2  # x2
#         new_bbx[2] = (bbx[2]+bbx[3])/2-(bbx[3]-bbx[2]) * subscale/2  # y1
#         new_bbx[3] = (bbx[2]+bbx[3])/2+(bbx[3]-bbx[2]) * subscale/2  # y2
#     elif action == 5:
#         new_bbx = bbx
#     return new_bbx


In [6]:
trans = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
    ])

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
feature_exactrator = torchvision.models.vgg16(
        pretrained=True).features.to(device)

c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
class DQN():
    """docstring for DQN"""

    def __init__(self, device):
        super().__init__()
        self.device = device
        self.eval_net, self.target_net = Q_zoom(), Q_zoom()
        self.eval_net.to(self.device)
        self.target_net.to(self.device)
        self.learn_step_counter = 0
        self.memory_counter = 0
        self.memory = np.zeros((MEMORY_CAPACITY, NUM_STATES * 2 + 2))
        # why the NUM_STATE*2 +2
        # When we store the memory, we put the state, action, reward and next_state in the memory
        # here reward and action is a number, state is a ndarray
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, state, EPISILO):
        state = torch.unsqueeze(torch.FloatTensor(state), 0).to(
            self.device)  # get a 1D array
        if np.random.randn() <= EPISILO:  # random policy
            action = np.random.randint(0, NUM_ACTIONS-1)
        else:  # greedy policy
            action_value = self.eval_net.forward(state)
            action = torch.max(action_value, 1)[1].cpu().item()
        return action

    def store_transition(self, state, action, reward, next_state):
        transition = np.hstack((state, [action, reward], next_state))
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):

        # update the parameters
        if self.learn_step_counter % Q_NETWORK_ITERATION == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch from memory
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        batch_memory = self.memory[sample_index, :]
        batch_state = torch.FloatTensor(
            batch_memory[:, :NUM_STATES]).to(self.device)
        batch_action = torch.LongTensor(
            batch_memory[:, NUM_STATES:NUM_STATES+1].astype(int)).to(self.device)
        batch_reward = torch.FloatTensor(
            batch_memory[:, NUM_STATES+1:NUM_STATES+2]).to(self.device)
        batch_next_state = torch.FloatTensor(
            batch_memory[:, -NUM_STATES:]).to(self.device)

        # q_eval
        q_eval = self.eval_net(batch_state).gather(1, batch_action)
        q_next = self.target_net(batch_next_state).detach()
        q_target_unterminated = batch_reward + GAMMA * \
            q_next.max(1)[0].view(BATCH_SIZE, 1)
        q_target = torch.where(
            batch_action != 5, q_target_unterminated, batch_reward)
        loss = self.loss_func(q_eval, q_target)
        print("step loss is {:.3f}".format(loss.cpu().detach().item()))

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    def load_eval_net(self, filepath):
        # Load the model state dict with the correct device mapping
        state_dict = torch.load(filepath, map_location=self.device)
        self.eval_net.load_state_dict(state_dict)
        self.eval_net.to(self.device)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dqn = DQN(device)
dqn.load_eval_net("eval_net8.pth")

In [11]:
EPISILO = -100

In [12]:
path_voc = r"e:\msrlOB-1\archive (4)\VOCtrainval_06-Nov-2007\VOC2007"

In [46]:
    # calculate the average IOU over test set
    image_names = np.array(load_images_names_in_data_set(
        'aeroplane_train', path_voc))
    single_plane_image_names = []
    single_plane_image_gts = []
    for image_name in image_names:
        annotation = get_bb_of_gt_from_pascal_xml_annotation(
            image_name, path_voc)
        #if (len(annotation) > 1):
        #    continue
        single_plane_image_names.append(image_name)
        single_plane_image_gts.append(annotation[0][1:])  # [[x1,x2,y1,y2] ...

    # save single_plane_image_names and single_plane_image_gts for future use
    np.save('single_plane_image_names_train.npy', single_plane_image_names)
    np.save('single_plane_image_gts_train.npy', single_plane_image_gts)

In [17]:
    # calculate the average IOU over test set
    image_names = np.array(load_images_names_in_data_set(
        'aeroplane_test', path_voc_test))
    single_plane_image_names = []
    single_plane_image_gts = []
    for image_name in image_names:
        annotation = get_bb_of_gt_from_pascal_xml_annotation(
            image_name, path_voc_test)
        #if (len(annotation) > 1):
        #    continue
        single_plane_image_names.append(image_name)
        single_plane_image_gts.append(annotation[0][1:])  # [[x1,x2,y1,y2] ...

    # save single_plane_image_names and single_plane_image_gts for future use
    np.save('single_plane_image_names.npy', single_plane_image_names)
    np.save('single_plane_image_gts.npy', single_plane_image_gts)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dqn = DQN(device)
dqn.load_eval_net("eval_net7.pth")

# random action

In [59]:
NUM_ACTIONS = 6
action = np.random.randint(0, NUM_ACTIONS)
print(action)

1


In [105]:
    single_plane_image_names= np.load('single_plane_image_names.npy')
    single_plane_image_gts= np.load('single_plane_image_gts.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc_test + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        print("start")
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                #action = dqn.choose_action(state, EPISILO)
                #action = np.random.randint(0, NUM_ACTIONS)
                action = 5
            #print(action)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_test)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        print(ma_iou)
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
start
0.41197058823529414
start
0.2516076294277929
start
0.0144
start
0.036864864864864864
start
0.39565333333333336
start
0.011434666666666666
start
0.812288
start
0.00792
start
0.06006006006006006
start
0.048938666666666665
start
0.8386726726726726
start
0.6250167224080267
start
0.09453089244851258
start
0.4681849315068493
start
0.24353153153153154
start
0.018582132564841498
start
0.008199288256227759
start
0.00651
start
0.45636636636636635
start
0.9936837209302326
start
0.37217081850533806
start
0.4824213333333333
start
0.6096533333333334
start
0.2384984984984985
start
0.08027733333333334
start
0.3790630630630631
start
0.2751711711711712
start
0.4196036036036036
start
0.021393442622950818
start
0.4132693333333333
start
0.00432
start
0.2621828908554572
start
0.8188125
start
0.8329973333333334
start
0.11634730538922156
start
0.0864
start
0.25282428115015976
start
0.34373573573573574
start
0.16104
start
0.251104
start
0.50701169

In [63]:
    single_plane_image_names= np.load('single_plane_image_names_train.npy')
    single_plane_image_gts= np.load('single_plane_image_gts_train.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                #action = dqn.choose_action(state, EPISILO)
                action = np.random.randint(0, NUM_ACTIONS)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
average IOU is 0.344


# default

In [103]:
    single_plane_image_names= np.load('single_plane_image_names_train.npy')
    single_plane_image_gts= np.load('single_plane_image_gts_train.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                action = dqn.choose_action(state, EPISILO)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        print(ma_iou)
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
0.6195238516682893
0.41770491803278686
0.5410380429753787
0.4588115431586476
0.40548095368594134
0.67208
0.40594561933534745
0.22105655381425568
0.40821770112680156
0.5544
0.8356586666666667
0
0.4091762950533066
0.4269778861036489
0.0006998225119753236
0.4285946435366299
0.5132661870503598
0.4053333333333333
0.5283963963963964
0.424457456302649
0.9504
0.6082656453852953
0
0.6016628873771731
0.7663783783783784
0.4322644322644323
0.4105742631918301
0.02693920979295595
0.5151808763079353
0.03800323192717903
0.4672797943968905
0.005839092703634799
0.38242642642642644
0
0
0.7050773333333333
0.4185836259122735
0.13887055631464224
0.45199157364585096
0.41724289214398835
0.5825375630371936
0.5535450463618851
0.5110048545461182
0
0.4715146666666667
0.4512955838707573
0.43959571620079735
0
0.541088
0.4736746666666667
0.7224092942100672
0.41521303703703705
0
0.4735196624981166
0.010416389580440123
0
0.4253067396096549
0
0.42368968532892987

In [15]:
    single_plane_image_names= np.load('single_plane_image_names.npy')
    single_plane_image_gts= np.load('single_plane_image_gts.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc_test + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.5:
                action = 5
            else:
                action = dqn.choose_action(state, EPISILO)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_test)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
average IOU is 0.238


# IOU = 0.4

In [19]:
    single_plane_image_names= np.load('single_plane_image_names.npy')
    single_plane_image_gts= np.load('single_plane_image_gts.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc_test + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                action = dqn.choose_action(state, EPISILO)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_test)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
average IOU is 0.385


In [28]:
    single_plane_image_names= np.load('single_plane_image_names.npy')
    single_plane_image_gts= np.load('single_plane_image_gts.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc_test + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]

        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                action = dqn.choose_action(state, EPISILO)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_test)
        ma_iou = 0
        for i in range(len(annotations)):
            cur_bbx = annotations[i][1:]
            ma_iou = max(ma_iou, cal_iou(bbx, cur_bbx))
        total_iou += ma_iou
        #total_iou += cal_iou(bbx, bbx_gt)
        #total_iou += ma_iou
        # write out bbx, bbx_gt to file
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

single_plane_image_names and single_plane_image_gts saved
average IOU is 0.383


# upper bound

In [64]:
import cv2

In [65]:
def selective_search(image):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    return rects


In [66]:

for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc + "\JPEGImages", image_name + ".jpg")
        image = cv2.imread(image_path)
        anh = image
        break

In [68]:
print(anh.shape)

(281, 500, 3)


In [71]:
trans = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
    ])

In [73]:
image_original = Image.open(image_path)
image = init_process(image_original, trans).to(device)

In [90]:
print(image.shape)

torch.Size([1, 3, 224, 224])


In [91]:
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from skimage import io
from skimage.measure import regionprops
from skimage.segmentation import felzenszwalb
import numpy as np

def selective_search(image):
    # Convert image from Tensor to NumPy array and adjust shape
    if isinstance(image, torch.Tensor):
        image = image.squeeze(0).permute(1, 2, 0).numpy()

    # Print the shape of the input image
    print(f"Processed image shape: {image.shape}")

    # Ensure the image has 3 channels
    if image.shape[2] != 3:
        raise ValueError("Image does not have 3 channels")

    # Apply segmentation
    img_lbl = felzenszwalb(image, scale=100, sigma=0.5, min_size=50)
    
    # Extract region properties
    regions = regionprops(img_lbl)
    
    rects = []
    for region in regions:
        minr, minc, maxr, maxc = region.bbox
        rects.append((minc, minr, maxc-minc, maxr-minr))
        
    return rects


In [100]:
def upper_bound_algorithm(ground_truth_annotations, region_proposals, max_steps):
    best_iou = 0
    # Tính toán IoU giữa các vùng đề xuất và các đối tượng thực địa
    for region in region_proposals:
        for obj in ground_truth_annotations:
            iou = cal_iou(region, obj)
            if iou > best_iou:
                best_iou = iou
    return best_iou

In [93]:
# Tạo các vùng đề xuất bằng Selective Search
rects = selective_search(image)

# Chuyển đổi các vùng đề xuất thành định dạng bounding box
rects_boxes = [(x, y, x + w, y + h) for (x, y, w, h) in rects]

# Tạo các đề xuất vùng cho thuật toán upper bound
max_steps = 100

Processed image shape: (224, 224, 3)


In [94]:
rects_boxes

[(145, 24, 224, 81),
 (117, 26, 148, 54),
 (0, 35, 15, 56),
 (19, 42, 59, 56),
 (209, 42, 224, 48),
 (99, 43, 147, 80),
 (202, 46, 224, 68),
 (213, 46, 224, 67),
 (0, 48, 108, 70),
 (179, 51, 209, 57),
 (135, 52, 150, 80),
 (59, 60, 109, 65),
 (179, 61, 187, 82),
 (52, 63, 110, 72),
 (56, 66, 109, 87),
 (101, 66, 114, 88),
 (0, 67, 20, 82),
 (0, 68, 92, 96),
 (23, 68, 52, 81),
 (199, 68, 224, 71),
 (45, 69, 87, 90),
 (170, 69, 185, 132),
 (183, 69, 198, 138),
 (198, 69, 212, 103),
 (188, 70, 197, 137),
 (112, 71, 138, 95),
 (197, 71, 198, 137),
 (199, 71, 224, 102),
 (115, 72, 128, 78),
 (167, 72, 179, 88),
 (193, 72, 199, 138),
 (86, 73, 93, 85),
 (114, 73, 129, 84),
 (189, 73, 193, 134),
 (199, 73, 207, 135),
 (0, 75, 14, 87),
 (203, 75, 212, 102),
 (61, 78, 92, 92),
 (29, 79, 50, 85),
 (136, 79, 160, 86),
 (147, 80, 168, 91),
 (124, 81, 158, 93),
 (169, 81, 177, 126),
 (167, 82, 175, 127),
 (70, 85, 105, 95),
 (98, 86, 113, 114),
 (114, 86, 156, 101),
 (122, 86, 168, 102),
 (88, 90,

In [ ]:
pip install opencv-contrib-python

   ---------------------------------------- 0.0/45.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/45.5 MB 660.6 kB/s eta 0:01:09
   ---------------------------------------- 0.1/45.5 MB 1.2 MB/s eta 0:00:40
   ---------------------------------------- 0.2/45.5 MB 1.6 MB/s eta 0:00:28
   ---------------------------------------- 0.2/45.5 MB 1.6 MB/s eta 0:00:28
   ---------------------------------------- 0.2/45.5 MB 876.1 kB/s eta 0:00:52
   ---------------------------------------- 0.2/45.5 MB 831.5 kB/s eta 0:00:55
   ---------------------------------------- 0.2/45.5 MB 808.4 kB/s eta 0:00:57
   ---------------------------------------- 0.2/45.5 MB 808.4 kB/s eta 0:00:57
   ---------------------------------------- 0.2/45.5 MB 684.7 kB/s eta 0:01:07
   ---------------------------------------- 0.3/45.5 MB 680.9 kB/s eta 0:01:07
   ---------------------------------------- 0.3/45.5 MB 655.2 kB/s eta 0:01:10
   ---------------------------------------- 0.3/45.5 MB 655.2 kB/s 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\21520\\anaconda3\\.conda\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



   ---------------------- ----------------- 25.1/45.5 MB 428.5 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 428.2 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 427.9 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 427.6 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 427.3 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 427.1 kB/s eta 0:00:48
   ---------------------- ----------------- 25.2/45.5 MB 427.1 kB/s eta 0:00:48
   ---------------------- ----------------- 25.3/45.5 MB 426.5 kB/s eta 0:00:48
   ---------------------- ----------------- 25.3/45.5 MB 426.2 kB/s eta 0:00:48
   ---------------------- ----------------- 25.3/45.5 MB 426.5 kB/s eta 0:00:48
   ---------------------- ----------------- 25.3/45.5 MB 426.2 kB/s eta 0:00:48
   ---------------------- ----------------- 25.4/45.5 MB 426.0 kB/s eta 0:00:48
   ---------------------- --------------

In [ ]:
    single_plane_image_names= np.load('single_plane_image_names.npy')
    single_plane_image_gts= np.load('single_plane_image_gts.npy')
    print("single_plane_image_names and single_plane_image_gts saved")
    # print out the average IOU
    total_iou = 0
    for index, image_name in enumerate(single_plane_image_names):
        image_path = os.path.join(
            path_voc_test + "\JPEGImages", image_name + ".jpg")
        image_original = Image.open(image_path)
        width, height = image_original.size
        bbx_gt = single_plane_image_gts[index]
        image = init_process(image_original, trans).to(device)
        bbx = [0, width, 0, height]
        history_action = np.zeros(his_actions*NUM_ACTIONS)
        with torch.no_grad():
            vector = feature_exactrator(
                image).cpu().detach().numpy().reshape(7*7*512)
        state = np.concatenate([history_action, vector])
        step = 0
        #ma_iou =  float("-inf")
        while (step < 10):
            iou = cal_iou(bbx, bbx_gt)
            #ma_iou = max(ma_iou, iou)
            if iou > 0.4:
                action = 5
            else:
                action = dqn.choose_action(state, EPISILO)
            new_bbx = update_bbx(bbx, action)
            if action == 5:
                break
            state = np.concatenate([history_action, vector])
            bbx = new_bbx
            step += 1
        annotations = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_test)
        anno = []
        IOU = upper_bound_algorithm(anno, rects_boxes, max_steps)
        total_iou +=IOU
        with open('bbx.txt', 'a') as f:
            f.write("bbx: {}, bbx_gt: {}\n".format(bbx, bbx_gt))
    print("average IOU is {:.3f}".format(
        total_iou/len(single_plane_image_names)))

In [ ]:
pip install scikit-image



   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.8 MB 1.7 MB/s eta 0:00:08
   ---------------------------------------- 0.1/12.8 MB 2.8 MB/s eta 0:00:05
   ---------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
   ---------------------------------------- 0.2/12.8 MB 1.1 MB/s eta 0:00:12
    --------------------------------------- 0.2/12.8 MB 980.4 kB/s eta 0:00:13
    --------------------------------------- 0.2/12.8 MB 827.9 kB/s eta 0:00:16
    --------------------------------------- 0.3/12.8 MB 785.2 kB/s eta 0:00:17
    --------------------------------------- 0.3/12.8 MB 785.2 kB/s eta 0:00:17
    --------------------------------------- 0.3/12.8 MB 710.0 kB/s eta 0:00:18
    --------------------------------------- 0.3/12.8 MB 631.2 kB/s eta 0:00:20
   - -------------------------------------- 0.3/12.8 MB 655.4 kB/s eta 0:00:20
   - -------------------------------------- 0.4/12.8 MB 636.5 kB/s e